# Drupal API Client

> Tools to interact with the Druapl REST API

In [ ]:
#| default_exp api

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
#| export
import requests
from requests.auth import HTTPBasicAuth
from tqdm import tqdm
from dotenv import load_dotenv
import os

In [ ]:
#| export
class DrupalAPIClient:
    """
    A client for interacting with a Drupal JSON:API endpoint, providing functionalities
    to fetch node IDs (nids) based on UUIDs and to delete nodes.

    Attributes:
        origin (str): The base URL for the Drupal site.
        auth (HTTPBasicAuth): Authentication credentials for the site.
        session (requests.Session): A requests session for making API calls.
    """

    @staticmethod
    def load_credentials(file_path):
        """
        Loads credentials from a .env file.

        Args:
            file_path (str): The path to the .env file.
        
        Returns:
            tuple: A tuple containing the URL, username, and password.
        """
        
        # dotenv
        load_dotenv(file_path)

        # get credentials

        url = os.getenv("DRUPAL_URL")
        username = os.getenv("DRUPAL_USERNAME")
        password = os.getenv("DRUPAL_PASSWORD")

        return url, username, password


    def __init__(self, origin, username, password):
        """
        Initializes the DrupalAPIClient with the necessary credentials and session.

        Args:
            origin (str): The base URL for the Drupal JSON:API.
            username (str): The username for authentication.
            password (str): The password for authentication.
        """
                
        self.origin = origin
        self.auth = HTTPBasicAuth(username, password)
        self.session = requests.Session()
        self.session.auth = self.auth  # Use auth for all requests in the session

    def get_nids(self, uuids, group_name, field_name, operator="IN"):
        """
        Retrieves node IDs (nids) for given UUIDs.

        Args:
            uuids (list of str): List of UUIDs to query for.
            group_name (str): The filter group name.
            field_name (str): The field name to match UUIDs against.
            operator (str, optional): The operator to use for matching. Default is "IN".

        Returns:
            dict: A dictionary mapping field item IDs to node IDs.
        """

        nids = {}
        for i in tqdm(range(0, len(uuids), 10)):
            nids.update(self._fetch_nids_batch(uuids[i:i+10], group_name, field_name, operator))
        return nids

    def _fetch_nids_batch(self, uuid_batch, group_name, field_name, operator):
        """
        Fetches a batch of node IDs from the Drupal JSON:API based on a subset of UUIDs.

        Args:
            uuid_batch (list of str): A batch of UUIDs.
            group_name (str): The filter group name.
            field_name (str): The field name to match UUIDs against.
            operator (str): The operator to use for matching.

        Returns:
            dict: A dictionary mapping field item IDs to node IDs for the batch.
        """

        params = self._build_filter_params(uuid_batch, group_name, field_name, operator)
        url = f'{self.origin}/jsonapi/node/assets'
        response = self.session.get(url, params=params)
        return self._parse_nids_response(response)

    def _build_filter_params(self, uuid_batch, group_name, field_name, operator):
        """
        Constructs the filter parameters for the API query.

        Args:
            uuid_batch (list of str): A batch of UUIDs to include in the filter.
            group_name (str): The filter group name.
            field_name (str): The field name to match UUIDs against.
            operator (str): The operator to use for matching.

        Returns:
            dict: A dictionary containing the constructed filter parameters.
        """

        params = {
            f"filter[{group_name}][condition][path]": field_name,
            f"filter[{group_name}][condition][operator]": operator
        }
        for idx, uuid in enumerate(uuid_batch):
            params[f"filter[{group_name}][condition][value][{idx}]"] = uuid
        return params

    def _parse_nids_response(self, response):
        """
        Parses the response from the API to extract node IDs.

        Args:
            response (requests.Response): The response object from the API.

        Returns:
            dict: A dictionary mapping field item IDs to node IDs.

        Raises:
            Exception: If the API response status is not 200 (OK).
        """

        if response.status_code != 200:
            raise Exception(f"Failed to fetch data: {response.status_code} - {response.text}")
        data = response.json()["data"]
        return {item["attributes"]["field_item_id"]: item["attributes"]["drupal_internal__nid"] for item in data}

    def delete(self, nid):
        """
        Deletes a node based on its node ID.

        Args:
            nid (int): The node ID of the node to be deleted.

        Returns:
            requests.Response: The response from the API after attempting to delete the node.
        """

        url = f'{self.origin}/node/{nid}'
        return self.session.delete(url)

    def delete_from_nids(self, nids):
        """
        Attempts to delete nodes based on a list of node IDs.

        Args:
            nids (dict): A dictionary mapping UUIDs to node IDs.

        Returns:
            list of dict: A list of dictionaries containing error information for each failed deletion.
        """

        errors = []
        for uuid, nid in tqdm(nids.items()):
            response = self.delete(nid)
            if response.status_code != 204:
                errors.append(self._handle_delete_error(uuid, nid, response))
        return errors

    def _handle_delete_error(self, uuid, nid, response):
        """
        Handles errors during the deletion process.

        Args:
            uuid (str): The UUID of the node.
            nid (int): The node ID.
            response (requests.Response): The response from the API.

        Returns:
            dict: A dictionary containing the error details.
        """

        return {
            "uuid": uuid,
            "nid": nid,
            "status_code": response.status_code,
            "message": response.text
        }

In [ ]:
show_doc(DrupalAPIClient)

---

### DrupalAPIClient

>      DrupalAPIClient (origin, username, password)

*A client for interacting with a Drupal JSON:API endpoint, providing functionalities
to fetch node IDs (nids) based on UUIDs and to delete nodes.

Attributes:
    origin (str): The base URL for the Drupal site.
    auth (HTTPBasicAuth): Authentication credentials for the site.
    session (requests.Session): A requests session for making API calls.*

In [ ]:
show_doc(DrupalAPIClient.__init__)

---

### DrupalAPIClient.__init__

>      DrupalAPIClient.__init__ (origin, username, password)

*Initializes the DrupalAPIClient with the necessary credentials and session.

Args:
    origin (str): The base URL for the Drupal JSON:API.
    username (str): The username for authentication.
    password (str): The password for authentication.*

In [ ]:
show_doc(DrupalAPIClient.get_nids)

---

### DrupalAPIClient.get_nids

>      DrupalAPIClient.get_nids (uuids, group_name, field_name, operator='IN')

*Retrieves node IDs (nids) for given UUIDs.

Args:
    uuids (list of str): List of UUIDs to query for.
    group_name (str): The filter group name.
    field_name (str): The field name to match UUIDs against.
    operator (str, optional): The operator to use for matching. Default is "IN".

Returns:
    dict: A dictionary mapping field item IDs to node IDs.*

In [ ]:
show_doc(DrupalAPIClient.delete)

---

### DrupalAPIClient.delete

>      DrupalAPIClient.delete (nid)

*Deletes a node based on its node ID.

Args:
    nid (int): The node ID of the node to be deleted.

Returns:
    requests.Response: The response from the API after attempting to delete the node.*

In [ ]:
show_doc(DrupalAPIClient.delete_from_nids)

---

### DrupalAPIClient.delete_from_nids

>      DrupalAPIClient.delete_from_nids (nids)

*Attempts to delete nodes based on a list of node IDs.

Args:
    nids (dict): A dictionary mapping UUIDs to node IDs.

Returns:
    list of dict: A list of dictionaries containing error information for each failed deletion.*

In [ ]:
show_doc(DrupalAPIClient.load_credentials)

---

### DrupalAPIClient.load_credentials

>      DrupalAPIClient.load_credentials (file_path)

*Loads credentials from a .env file.

Args:
    file_path (str): The path to the .env file.

Returns:
    tuple: A tuple containing the URL, username, and password.*

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()